<a href="https://colab.research.google.com/github/Sairaj-97/Deep-Learning/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow import keras

In [ ]:
(ds_train,ds_test),ds_info = tfds.load(
    'tf_flowers',
    split=['train[:80%]','train[80%:]'],
    with_info=True,
    as_supervised=True
)

In [ ]:
print(ds_info)

In [ ]:
for image, label in ds_train.take(1):
    print("Image shape:", image.shape)
    print("Label:", label)

In [5]:
# Preprocessing function for resizing and normalizing images
def preprocess(image, label):
    # Resize the input image to 128x128 pixels
    # (This ensures all images have the same shape for CNN input)
    image = tf.image.resize(image, (128, 128))

    # Convert image data type from uint8 (0–255) to float32
    image = tf.cast(image, tf.float32)

    # Normalize pixel values from [0, 255] to [0, 1]
    # (This helps the model train faster and perform better)
    image = image / 255.0

    # Return the processed image and its corresponding label
    return image, label


In [ ]:
# Apply the preprocessing function to each (image, label) pair in the training dataset
# 'map' applies the function and 'num_parallel_calls=AUTOTUNE' uses multiple CPU threads to speed it up
ds_train = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# Prefetching overlaps data preprocessing and model execution for better performance
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

# Apply the same preprocessing steps to the test dataset
ds_test = ds_test.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# Prefetch the test data as well to ensure smooth evaluation
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

for image, label in ds_train.take(1):
    print("Image shape:", image.shape)
    print("Label:", label)


#batching is mandatory when using model.fit() in TensorFlow/Keras.



In [7]:
ds_train = ds_train.batch(32)
ds_test = ds_test.batch(32)

#loading existing trained model

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(128, 128, 3),
                                               include_top=False,
                                               weights='imagenet')

#freeze the training of this base model

In [9]:
base_model.trainable=False

#making our dense layers and attaching the pretrained model

In [14]:
inputs = keras.Input(shape=(128, 128, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.MaxPool2D()(x)
# Flatten the output before the dense layer
x = keras.layers.Flatten()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(5,activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [15]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model.fit(ds_train,epochs=5)

In [18]:
loss,accuracy=model.evaluate(ds_test)

23/23 ━━━━━━━━━━━━━━━━━━━━ 8s 201ms/step - accuracy: 0.8667 - loss: 0.5682


In [ ]:
print(accuracy)